In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import modules :

In [ ]:
import time
import datatable as dt
import pandas as pd
import matplotlib.pyplot as plt
import catboost
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import optuna
# import imblearn
#from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.metrics import geometric_mean_score
from imblearn.metrics import make_index_balanced_accuracy
import seaborn as sns

In [ ]:
RANDOM_STATE = 42
N_SPLITS = 20
CLIP_FEATURES = False
CLUSTER_DISTANCE = False

## Import data :

In [ ]:
%%time
train_file_name = "/kaggle/input/tabular-playground-series-dec-2021/train.csv"
test_file_name = "/kaggle/input/tabular-playground-series-dec-2021/test.csv"
submission_file_name = "/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv"
train_dt = dt.fread(train_file_name)
test_dt = dt.fread(test_file_name)
submission = pd.read_csv(submission_file_name)    

In [ ]:
def initialize():
    train_file_name = "/kaggle/input/tabular-playground-series-dec-2021/train.csv"
    test_file_name = "/kaggle/input/tabular-playground-series-dec-2021/test.csv"
    submission_file_name = "/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv"
    train_dt = dt.fread(train_file_name)
    test_dt = dt.fread(test_file_name)
    submission = pd.read_csv(submission_file_name)
    
    X = train_dt.to_pandas()
    X_test = test_dt.to_pandas()
    
    X = reduce_mem_usage(X)
    X_test = reduce_mem_usage(X_test)
    return X, X_test

### DATA FIELDS :  

(from https://www.kaggle.com/c/forest-cover-type-prediction/overview)

Elevation - Elevation in meters

Aspect - Aspect in degrees azimuth

Slope - Slope in degrees

Horizontal_Distance_To_Hydrology - Horz Dist to nearest surface water features

Vertical_Distance_To_Hydrology - Vert Dist to nearest surface water features

Horizontal_Distance_To_Roadways - Horz Dist to nearest roadway

Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice

Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice

Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice

Horizontal_Distance_To_Fire_Points - Horz Dist to nearest wildfire ignition points

Wilderness_Area (4 binary columns, 0 = absence or 1 = presence) - Wilderness area
designation

Soil_Type (40 binary columns, 0 = absence or 1 = presence) - Soil Type designation

Cover_Type (7 types, integers 1 to 7) - Forest Cover Type designation



The wilderness areas are:

1 - Rawah Wilderness Area

2 - Neota Wilderness Area

3 - Comanche Peak Wilderness Area

4 - Cache la Poudre Wilderness Area



The soil types are:

1 Cathedral family - Rock outcrop complex, extremely stony.

2 Vanet - Ratake families complex, very stony.

3 Haploborolis - Rock outcrop complex, rubbly.

4 Ratake family - Rock outcrop complex, rubbly.

5 Vanet family - Rock outcrop complex complex, rubbly.

6 Vanet - Wetmore families - Rock outcrop complex, stony.

7 Gothic family.

8 Supervisor - Limber families complex.

9 Troutville family, very stony.

10 Bullwark - Catamount families - Rock outcrop complex, rubbly.

11 Bullwark - Catamount families - Rock land complex, rubbly.

12 Legault family - Rock land complex, stony.

13 Catamount family - Rock land - Bullwark family complex, rubbly.

14 Pachic Argiborolis - Aquolis complex.

15 unspecified in the USFS Soil and ELU Survey.

16 Cryaquolis - Cryoborolis complex.

17 Gateview family - Cryaquolis complex.

18 Rogert family, very stony.

19 Typic Cryaquolis - Borohemists complex.

20 Typic Cryaquepts - Typic Cryaquolls complex.

21 Typic Cryaquolls - Leighcan family, till substratum complex.

22 Leighcan family, till substratum, extremely bouldery.

23 Leighcan family, till substratum - Typic Cryaquolls complex.

24 Leighcan family, extremely stony.

25 Leighcan family, warm, extremely stony.

26 Granile - Catamount families complex, very stony.

27 Leighcan family, warm - Rock outcrop complex, extremely stony.

28 Leighcan family - Rock outcrop complex, extremely stony.

29 Como - Legault families complex, extremely stony.

30 Como family - Rock land - Legault family complex, extremely stony.

31 Leighcan - Catamount families complex, extremely stony.

32 Catamount family - Rock outcrop - Leighcan family complex, extremely stony.

33 Leighcan - Catamount families - Rock outcrop complex, extremely stony.

34 Cryorthents - Rock land complex, extremely stony.

35 Cryumbrepts - Rock outcrop - Cryaquepts complex.

36 Bross family - Rock land - Cryumbrepts complex, extremely stony.

37 Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.

38 Leighcan - Moran families - Cryaquolls complex, extremely stony.

39 Moran family - Cryorthents - Leighcan family complex, extremely stony.

40 Moran family - Cryorthents - Rock land complex, extremely stony.

In [ ]:
#Setting up options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.3f}".format

In [ ]:
X = train_dt.to_pandas()
X = train_dt.to_pandas()
X_test = test_dt.to_pandas()
target = 'Cover_Type'

X_id = X.Id
X_test_id = X_test.Id

In [ ]:
num_features = [col for col in X.columns if X[col].dtype in ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']]
cat_features = [col for col in X.columns if X[col].dtype in ['bool']]

## Reduce memory

In [ ]:
# will do 83.9% reduction im memory usage
def reduce_mem_usage(df, verbose=True):#from santander ml comp
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns :
        if df[col].dtype=='bool' :
            df[col] = df[col].astype(int)
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
X = reduce_mem_usage(X)
X_test = reduce_mem_usage(X_test)

## Feature engineering :

In [ ]:
%%time
X, X_test = initialize()

In [ ]:
def feature_engineering(X, X_test, clip_features=CLIP_FEATURES, cluster_distance=CLUSTER_DISTANCE):
    """
    X : train data set
    X_test : test data set
    clip_features : bool : clip features to naturale scale or handel them by other way : abs or rescaling
    cluster_distance : bool : True if you want to add cluster_distance features else false
    (do not improuve the score on kaggle comp with catboost. See with other models...)
    return : X (modified, target droped), y : target, X_test (modified)
    
    """
    
    # drop the only one sample with Cover_Type == 5 :
    outlier=X[X.Cover_Type==5].index
    X.drop(index=outlier, axis=1, inplace=True)
    X.reset_index(inplace=True, drop=True)
    print('Cover_Type 5 droped : just one sample')
    
    y = X.Cover_Type.astype(int)
    X_test.drop(['Id'], axis=1, inplace=True)
    X.drop(['Cover_Type', 'Id'], axis=1, inplace=True)
    print('Target feature y created')
    print('Features "Id" droped in train and test set. "Cover_type" droped in train set')
    
    
    
    # drop 'Soil_Type7' and 'Soil_Type15' : no sample
    X.drop(['Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)
    X_test.drop(['Soil_Type7', 'Soil_Type15'], axis=1, inplace=True)
    print('"Soil_Type7" and "Soil_Type15" features droped in train and test set : no samples')
    
    
    # sum of soil type
    soil_features=[f'Soil_Type{i}' for i in range(1,41) if not i in (7, 15) ]
    X['sum_soil']=X[soil_features].sum(axis=1)
    X_test['sum_soil']=X_test[soil_features].sum(axis=1)
    print('feature "sum_soil" added')
    
    # Handle distance features, Aspect and Hillshade_features, wich are out of natural range : clip or else
    # Aspect is supposed to be between 0 and 360
    # Hillshade is supposed to be between 0 and 255 
    # --> http://www.geography.hunter.cuny.edu/~jochen/gtech361/lectures/lecture11/concepts/hillshade.htm
    # distance features are supposed to be positive
    
    distance_features = ['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 
                             'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points']
    Hillshade_features = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
    
    if clip_features:
        # clip values for "Aspect" between 0° and 360°
        X['Aspect'].clip(0, 360, inplace=True)
        X_test['Aspect'].clip(0, 360, inplace=True)
        print('Feature "Aspect" cliped between 0 and 360 in train and test set')
    
        # clip values for "Hillshade_features" between 0 and 255
        for feature in Hillshade_features:
            X[feature].clip(0, 255, inplace=True)
            X_test[feature].clip(0, 255, inplace=True)
        print('Features "Hillshade" cliped positive in train and test set')
            
        # clip values for distances >0
        for feature in distance_features:
            X[feature].clip(0, inplace=True)
            X_test[feature].clip(0, inplace=True)
        print('Distance features cliped positive in train and test set')
        
        X['Slope'].clip(0, inplace=True)
        X_test['Slope'].clip(0, inplace=True)
        print('"Slope" features cliped positive in train and test set')
        
    else :
        
        #X["Aspect"][X["Aspect"] < 0] += 360
        #X["Aspect"][X["Aspect"] > 359] -= 360
        X["Aspect"] = X["Aspect"].map(lambda x : x + 360 if x < 0 else (x- 360 if x> 359 else x))
        
        #X_test["Aspect"][X_test["Aspect"] < 0] += 360
        #X_test["Aspect"][X_test["Aspect"] > 359] -= 360
        X_test["Aspect"] = X_test["Aspect"].map(lambda x : x + 360 if x < 0 else (x- 360 if x> 359 else x))
        print('Features "Aspect" rescaled between 0 and 360 in train and test set')
        
        for feature in distance_features:
            X[feature] = np.abs(X[feature])
            X_test[feature] = np.abs(X_test[feature])
        print('Distance features rescaled positive in train and test set')
        
        X["Slope"] = np.abs(X["Slope"])
        X_test["Slope"] = np.abs(X_test["Slope"])
        print('"Slope" feature rescaled positive in train and test set')
        
        for feature in Hillshade_features:
            X[feature] = X[feature].map(lambda x : -x if x < 0 else (x - (x - 255) if x> 255 else x))
            X_test[feature] = X_test[feature].map(lambda x : -x if x < 0 else (x - (x - 255) if x> 255 else x))
            
        print('Features "Hillshade" rescaled in [0, 255] in train and test set')
        
    
    # sum of Wilderness_Area features :
    Wilderness_Area_features = [f'Wilderness_Area{i}' for i in range(1,5)]
    X['sum_W_A']=X[Wilderness_Area_features].sum(axis=1)
    X_test['sum_W_A']=X_test[Wilderness_Area_features].sum(axis=1)
    print('feature "sum_W_A" added in train and test set')
    
    # Hillshade_features : mean and amplitude :
    X['mean_Hillshade']=X[Hillshade_features].mean(axis=1)
    X_test['mean_Hillshade']=X_test[Hillshade_features].mean(axis=1)
    X['amp_Hillshade']=X[Hillshade_features].max(axis=1)-X[Hillshade_features].min(axis=1)
    X_test['amp_Hillshade']=X_test[Hillshade_features].max(axis=1) - X_test[Hillshade_features].min(axis=1)
    print('feature "amp_Hillshade" added in train and test set')
    
    # Euclidean distance to hydrology
    X['dist_water']=np.sqrt(
        np.square(X['Horizontal_Distance_To_Hydrology'].astype('int64')) + np.square(X['Vertical_Distance_To_Hydrology'].astype('int64'))
            )
    X_test['dist_water']=np.sqrt(
        np.square(X_test['Horizontal_Distance_To_Hydrology'].astype('int64')) + np.square(X_test['Vertical_Distance_To_Hydrology'].astype('int64'))
            )
    print('feature "dist_water" added in train and test set')
    
    # Add cluster distances features : n_cluster =3 (best results after optuna tuning)
    if cluster_distance :
        # Add "cluster distance to centroids" features
        n_clusters = 3
        kmeans=KMeans(n_clusters=n_clusters)
    
        cluster_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
           'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
           'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
           'Horizontal_Distance_To_Fire_Points', 'mean_Hillshade', 'amp_Hillshade', 'dist_water']
        X_test_scaled = X_test.loc[:, cluster_features]
        X_test_scaled = (X_test_scaled - X_test_scaled.mean(axis=0)) / X_test_scaled.std(axis=0)
        X_scaled = X.loc[:, cluster_features]
        X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
        X_cd = kmeans.fit_transform(X_scaled)
        X_test_cd = kmeans.transform(X_test_scaled)

        X_cd = pd.DataFrame(X_cd, index =X.index,  columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
        X = X.join(X_cd)
        X_test_cd = pd.DataFrame(X_test_cd, index =X_test.index, columns=[f"Centroid_{i}" for i in range(X_test_cd.shape[1])])
        X_test = X_test.join(X_test_cd)
        print('Feature cluster distance added in train and test set')
    print('Feature engineering done')
    
    return X, y, X_test

In [ ]:
X, y, X_test = feature_engineering(X, X_test, clip_features=CLIP_FEATURES, cluster_distance=CLUSTER_DISTANCE)

In [ ]:
X.head(5)

In [ ]:
X.describe()

In [ ]:
feature='Horizontal_Distance_To_Roadways'
# Horizontal_Distance_To_Fire_Points
sns.displot(X[feature], kde=True)
plt.show()

In [ ]:
feature='Horizontal_Distance_To_Fire_Points'
sns.displot(X[feature], kde=True)
plt.show()

In [ ]:
feature='Slope'
# Horizontal_Distance_To_Fire_Points
sns.displot(X[feature], kde=True)
plt.show()

In [ ]:
from scipy.stats import chi2_contingency
tableW_A = pd.crosstab(X['sum_W_A'], y)
testW_A=chi2_contingency(tableW_A)
testW_A

In [ ]:
tableSoil = pd.crosstab(X['sum_soil'], y)
testSoil=chi2_contingency(tableSoil)
testSoil

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


In [ ]:
discrete_features = [col for col in X.columns if X.loc(axis=1)[col].dtypes != 'float64']
discrete_features = [col in discrete_features for col in X.columns]

In [ ]:
# discrete_features

In [ ]:
# mi_scores = make_mi_scores(X, y, discrete_features)


#### MI Scores : 
Elevation                            0.618  
Wilderness_Area4                     0.066  
Horizontal_Distance_To_Roadways      0.033  
Horizontal_Distance_To_Fire_Points   0.022  
Wilderness_Area1                     0.019  
sum_soil                             0.013  
sum_W_A                              0.011  
Wilderness_Area3                     0.011  
Soil_Type39                          0.006  
Soil_Type10                          0.006  
Soil_Type38                          0.004  
Vertical_Distance_To_Hydrology       0.004  
Soil_Type6                           0.004  
Soil_Type3                           0.003  
Soil_Type40                          0.003  
Wilderness_Area2                     0.003  
Soil_Type2                           0.003  
Soil_Type23                          0.002  
dist_water                           0.002  
Soil_Type32                          0.002  
mean_Hillshade                       0.002  
Horizontal_Distance_To_Hydrology     0.002  
Soil_Type33                          0.002  
Soil_Type22                          0.001  
Soil_Type29                          0.001  
Slope                                0.001  
Soil_Type30                          0.001  
Soil_Type35                          0.001  
Soil_Type4                           0.001  
Soil_Type31                          0.001  
Soil_Type13                          0.001  
Soil_Type24                          0.001  
Soil_Type37                          0.001  
Soil_Type11                          0.000  
Soil_Type17                          0.000  
Hillshade_3pm                        0.000  
Soil_Type36                          0.000  
amp_Hillshade                        0.000  
Aspect                               0.000  
Soil_Type12                          0.000  
Soil_Type1                           0.000  
Hillshade_9am                        0.000  
Hillshade_Noon                       0.000  
Soil_Type20                          0.000  
Soil_Type19                          0.000  
Soil_Type34                          0.000  
Soil_Type14                          0.000  
Soil_Type18                          0.000  
Soil_Type16                          0.000  
Soil_Type26                          0.000  
Soil_Type21                          0.000  
Soil_Type27                          0.000  
Soil_Type28                          0.000  
Soil_Type9                           0.000  
Soil_Type25                          0.000  
Soil_Type5                           0.000  
Soil_Type8                           0.000

In [ ]:
# mi_scores  
# show a few features with their MI scores

In [ ]:
# Wilderness_Area_features :
for i in range(1,5) :
    tableWilderness = pd.crosstab(X[f'Wilderness_Area{i}'], y)
    print(f'Wilderness_Area{i} :\n')
    display(tableWilderness)

In [ ]:
X_test.head(5)

In [ ]:
# Can't be done : too long
#%%time
#sm = SMOTE(random_state=RANDOM_STATE, n_jobs=-1)
#X, y = sm.fit_resample(X, y)

In [ ]:
X.shape

In [ ]:
X.to_pickle('../input/tabular-playground-series-dec-2021/X_resample.pkl')
y.to_pickle('../input/tabular-playground-series-dec-2021/y_resample.pkl')
# df = pd.read_pickle(file_name)

In [ ]:
test_preds=[]
count=0   

alpha = 0.1
geo_mean = make_index_balanced_accuracy(alpha=alpha, squared=True)(geometric_mean_score)

def objective(trial):
    
    kf = StratifiedShuffleSplit(n_splits=N_SPLITS, test_size=0.2, random_state=RANDOM_STATE)
    score=[]
    score_geo=[]
    global X_test
    global count
    idx=0
    
    
    n_clusters = trial.suggest_int('n_clusters', 2, 10)
    kmeans=KMeans(n_clusters=n_clusters)
    
    cluster_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
           'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
           'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
           'Horizontal_Distance_To_Fire_Points', 'mean_Hillshade', 'amp_Hillshade', 'dist_water']
        
    
    idx=0
    
    for train_index, test_index in kf.split(X, y) :
        
        start = time.time()
        
        
        X_train,  y_train = X.iloc[train_index], y.iloc[train_index]
        X_valid, y_valid = X.iloc[test_index], y.iloc[test_index]
        print("smote done")       
       
        # Standardize
        X_test_in = X_test.copy()
        X_train_scaled = X_train.loc[:, cluster_features]
        X_train_scaled = (X_train_scaled - X_train_scaled.mean(axis=0)) / X_train_scaled.std(axis=0)
        X_valid_scaled = X_valid.loc[:, cluster_features]
        X_valid_scaled = (X_valid_scaled - X_valid_scaled.mean(axis=0)) / X_valid_scaled.std(axis=0)
        X_test_scaled = X_test_in.loc[:, cluster_features]
        X_test_scaled = (X_test_scaled - X_test_scaled.mean(axis=0)) / X_test_scaled.std(axis=0)
        #X_test_scaled = X_test_in.loc[:, cluster_features]
        #X_test_scaled = (X_test_scaled - X_test_scaled.mean(axis=0)) / X_test_scaled.std(axis=0)
        
        
        X_train_cd = kmeans.fit_transform(X_train_scaled)
        X_valid_cd = kmeans.transform(X_valid_scaled)
        # X_test_cd = kmeans.transform(X_test_scaled)
        
        # Label features and join to dataset
        X_train_cd = pd.DataFrame(X_train_cd, index =X_train.index,  columns=[f"Centroid_{i}" for i in range(X_train_cd.shape[1])])
        X_train = X_train.join(X_train_cd)
        X_valid_cd = pd.DataFrame(X_valid_cd, index =X_valid.index, columns=[f"Centroid_{i}" for i in range(X_valid_cd.shape[1])])
        X_valid = X_valid.join(X_valid_cd)
        X_test_cd = kmeans.transform(X_test_scaled)
        X_test_cd = pd.DataFrame(X_test_cd, index =X_test_in.index, columns=[f"Centroid_{i}" for i in range(X_test_cd.shape[1])])
        X_test_in = X_test_in.join(X_test_cd)
        
        # cat_clf = make_pipeline(SMOTE(random_state=RANDOM_STATE), 
        #                            CatBoostClassifier(iterations=1000, task_type="GPU")) 
        cat_clf = catboost.CatBoostClassifier(iterations=2000, task_type="GPU")
        cat_clf.fit(X_train, y_train, verbose=0) #eval_set=[X_valid, y_valid]
        cat_preds = cat_clf.predict(X_valid)
        cat_test_preds = cat_clf.predict(X_test_in)
        test_preds.append(cat_test_preds)
        score.append(accuracy_score(cat_preds, y_valid))
        score_geo.append(geo_mean(cat_preds, y_valid))
        end = time.time()
        idx+=1
        print(f'Finished round{count}, fold {idx} done in {end-start} with accuracy {accuracy_score(cat_preds, y_valid)} and geo_score {geo_mean(cat_preds, y_valid)}  ')
    count+=1    
    print(f'Round {count} finished with score {np.mean(score)} and geo_score {np.mean(score_geo)}')
    return np.mean(score_geo)
        


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

In [ ]:
# Add "cluster distance to centroids" features
n_clusters = 3
kmeans=KMeans(n_clusters=n_clusters)
    
cluster_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
           'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
           'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
           'Horizontal_Distance_To_Fire_Points', 'mean_Hillshade', 'amp_Hillshade', 'dist_water']
X_test_scaled = X_test.loc[:, cluster_features]
X_test_scaled = (X_test_scaled - X_test_scaled.mean(axis=0)) / X_test_scaled.std(axis=0)
X_scaled = X.loc[:, cluster_features]
X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
X_cd = kmeans.fit_transform(X_scaled)
X_test_cd = kmeans.transform(X_test_scaled)

X_cd = pd.DataFrame(X_cd, index =X.index,  columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])])
X = X.join(X_cd)
X_test_cd = pd.DataFrame(X_test_cd, index =X_test.index, columns=[f"Centroid_{i}" for i in range(X_test_cd.shape[1])])
X_test = X_test.join(X_test_cd)

    
 

In [ ]:
X.head(5)

In [ ]:
X_test.head(5)

In [ ]:
print(study.best_trial)
print(study.best_params)

FrozenTrial(number=1, values=[0.9606684999999999], datetime_start=datetime.datetime(2021, 12, 5, 10, 39, 14, 808230), datetime_complete=datetime.datetime(2021, 12, 5, 10, 46, 21, 260684), params={'n_clusters': 3}, distributions={'n_clusters': IntUniformDistribution(high=10, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)
{'n_clusters': 3}

In [ ]:
'''
kf = StratifiedShuffleSplit(n_splits=N_SPLITS, test_size=0.2, random_state=RANDOM_STATE)
X1 = pd.concat([X, X.iloc[y[y==5].index]], axis=0)
y1 = pd.concat([y, y[y==5]], axis=0)
for train_index, test_index in kf.split(X1, y1.astype(int)) :
    X_train = X1.iloc[train_index]
    y_train = y1.iloc[train_index]
    X_valid = X1.iloc[test_index]
    y_valid = y1.iloc[test_index]
    
    n_clusters = 4
        
    cluster_features = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
           'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
           'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
           'Horizontal_Distance_To_Fire_Points', 'mean_Hillshade', 'amp_Hillshade', 'dist_water']
        
    kmeans=KMeans(n_clusters=n_clusters)
    # Standardize
    X_train_scaled = X_train.loc[:, cluster_features]
    X_train_scaled = (X_train_scaled - X_train_scaled.mean(axis=0)) / X_train_scaled.std(axis=0)
    X_valid_scaled = X_valid.loc[:, cluster_features]
    X_valid_scaled = (X_valid_scaled - X_valid_scaled.mean(axis=0)) / X_valid_scaled.std(axis=0)
    
    X_train_cd = kmeans.fit_transform(X_train_scaled)
    X_valid_cd = kmeans.transform(X_valid_scaled)
    print('X_train_cd :')
    display(X_train_cd[:5])
    print('X_train_cd.shape :', X_train_cd.shape)
    
    print('----------------')
    print('X_valid_cd :')
    display(X_valid_cd[:5])
    print('X_valid_cd.shape :', X_valid_cd.shape)
    
    # Label features and join to dataset
    X_train_cd = pd.DataFrame(X_train_cd, index =X_train.index,  columns=[f"Centroid_{i}" for i in range(X_train_cd.shape[1])])
    X_train = X_train.join(X_train_cd)
    X_valid_cd = pd.DataFrame(X_valid_cd, index =X_valid.index, columns=[f"Centroid_{i}" for i in range(X_valid_cd.shape[1])])
    display(X_valid_cd.head(5))
    X_valid = X_valid.join(X_valid_cd)
    print('--------------------------')
    print('X_train :')
    display(X_train.head(5))
    print('--------------------------')
    print('X_valid :')
    display(X_valid.head(5))
'''

In [ ]:
X.isnull().sum()

## Modeling :

In [ ]:
cat_clf = catboost.CatBoostClassifier(iterations=2000, task_type="GPU")

In [ ]:
final_test_predictions = [] # list of predictions for X_test on every split
final_valid_predictions = {} # key : index of validation set, values : predictions for X_valid on every split (probabilities)
scores = []
score_geo=[]

kf = StratifiedShuffleSplit(n_splits=N_SPLITS, test_size=0.2, random_state=RANDOM_STATE)
alpha = 0.1
geo_mean = make_index_balanced_accuracy(alpha=alpha, squared=True)(geometric_mean_score)

for fold, (train_idx, valid_idx) in enumerate(kf.split(X = X, y = y)):
    start = time.time()
    print(10*"=", f"Fold={fold}", 10*"=")

    x_train = X.loc[train_idx, :]
    x_valid = X.loc[valid_idx, :]
    y_train = y[train_idx]
    y_valid = y[valid_idx]
    cat_clf = catboost.CatBoostClassifier(iterations=2000, task_type="GPU")
    
    cat_clf.fit(x_train, y_train, verbose=0)

    preds_valid = cat_clf.predict(x_valid)
    final_valid_predictions.update(dict(zip(valid_idx, preds_valid)))
    
    accur = accuracy_score(y_valid, preds_valid)
    geo_score = geo_mean(y_valid, preds_valid)
    scores.append(accur)
    score_geo.append(geo_score)
    #loss = log_loss(y_valid, preds_valid)
    print('accuracy: ', accur)
    print('geo_score', geo_score)
    
    
    
    test_preds = cat_clf.predict(X_test)
    final_test_predictions.append(test_preds)
    end = time.time()
    print(f"Finished fold {fold} in {end - start}")
 

In [ ]:
print(f"scores -> mean: {np.mean(scores)}, std: {np.std(scores)}")
print(f"scores_geo -> mean: {np.mean(score_geo)}, std: {np.std(score_geo)}")

In [ ]:
# dict values : rows of the df
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("train_pred_1.csv", index=False)

In [ ]:
df = pd.DataFrame(np.column_stack(final_test_predictions))

df.head(15)

In [ ]:
# mod = df.mode(axis=1)

In [ ]:
# mod2 = mod.loc[:,0].astype(int) 

In [ ]:
# mod2.head(10)

In [ ]:
#col=mod.loc[:5,0].astype(int)
#col

In [ ]:
#df['mode']=mod

In [ ]:
#df_test=df.loc[70:80]
#df_test

In [ ]:
mod2=df_test.mode(axis=1)
df_test['mode']=mod2
df_test.head(5)

In [ ]:
df.head(100)

In [ ]:
submission['Cover_Type'] = mod2

submission.to_csv("test_pred_1.csv",index=None)
submission.to_csv("catboost_cv_FE.csv",index=None)
submission[70:79]

In [ ]:
submission[70:79]

In [ ]:
print('done')

In [ ]:
import shap
from catboost import *
explainer = shap.TreeExplainer(cat_clf)
shap_values = explainer.shap_values(X)

In [ ]:
# visualize the first prediction's explanation
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X)

In [ ]:
cat_preds = cat_clf.predict(X_test)

In [ ]:
submission['Cover_Type'] = cat_preds
submission.head(5)

In [ ]:
submission.to_csv("submission.csv", index=None)

V1 : without FE:
Kaggle score : 0.95310

V2 : with FE : 
Kaggle score : 0.95460

V3 : with cv, FE and cluster distance : 0.95466

V4 : with cv, FE without cluster distance : 0.95484

V5 : with cv no FE : 0.95397
